# Load Model (Colab version)

Load persisted model and run experiments

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Get the RAPIDS-Colab install files
!git clone https://github.com/rapidsai-community/rapidsai-csp-utils.git

# Install RAPIDS via pip (default: stable version suited for Colab)
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 621, done.
remote: Counting objects: 100% (187/187), done.
remote: Compressing objects: 100% (102/102), done.
remote: Total 621 (delta 143), reused 86 (delta 85), pack-reused 434 (from 3)
Receiving objects: 100% (621/621), 205.72 KiB | 18.70 MiB/s, done.
Resolving deltas: 100% (317/317), done.
Installing RAPIDS remaining 25.10 libraries
Using Python 3.12.12 environment at: /usr
Resolved 175 packages in 8.81s
Prepared 18 packages in 12.88s
Uninstalled 11 packages in 171ms
Installed 18 packages in 43ms
 - bokeh==3.7.3
 + bokeh==3.6.3
 + cucim-cu12==25.10.0
 + cugraph-cu12==25.10.1
 + cuxfilter-cu12==25.10.0
 + datashader==0.18.2
 - holoviews==1.22.1
 + holoviews==1.20.2
 + jupyter-server-proxy==4.4.0
 - nvidia-cublas-cu12==12.6.4.1
 + nvidia-cublas-cu12==12.9.1.4
 - nvidia-cuda-nvcc-cu12==12.5.82
 + nvidia-cuda-nvcc-cu12==12.9.86
 - nvidia-cuda-nvrtc-cu12==12.6.77
 + nvidia-cuda-nvrtc-cu12==12.9.86
 - nvidia-cufft-cu12==11

In [4]:
import cuml.accel
cuml.accel.install()    # enable zero-code-change acceleration

In [5]:
from cuml import accel
accel.enabled()

True

In [6]:
# Install required packages if needed
!pip install rdp fastdtw pactus fvcore timm "yupi==0.12.5" | grep -v 'already satisfied'

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.9/63.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 115.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [7]:
# form symbolic link with traj_xai codebase
!ln -s "/content/drive/MyDrive/XAI4Traj/traj_xai" "/content/traj_xai"

In [8]:
import sys
sys.path.append('/content/traj_xai')

In [9]:
import os
from joblib import load
from pactus import Dataset

from traj_xai.src import (
    mdl_segmentation,
    rdp_segmentation,
    sliding_window_segmentation,
    gaussian_perturbation,
    scaling_perturbation,
    rotation_perturbation,
    gan_perturbation,
    run_experiments
)

In [21]:
SEED = 42
# SEED = None

datasets = [
    Dataset.animals,  # 0
    Dataset.geolife,  # 1
    Dataset.hurdat2,  # 2
    Dataset.uci_pen_digits,  # 3
    Dataset.uci_movement_libras,  # 4
    Dataset.uci_characters,  # 5
    Dataset.mnist_stroke  # 6
]

# Load dataset
dataset_idx = 1
dataset = datasets[dataset_idx]()
print(f"Dataset loaded: {len(dataset.trajs)} trajectories ({datasets[dataset_idx].__name__})")

# Split data into train and test subsets
train, test = dataset.split(0.8, random_state=SEED)
print(f"Train set: {len(train.trajs)} trajectories")
print(f"Test set: {len(test.trajs)} trajectories")

[==================================================] 100.00%
Dataset loaded: 9288 trajectories (geolife)
Train set: 7430 trajectories
Test set: 1858 trajectories


In [22]:
model_idx = 2
#             0      1      2       3          4             5
model_str = ["rf", "knn", "lstm", "gru", "transformer", "trajformer"][model_idx]
dataset_str = datasets[dataset_idx].__name__
model_dir = "drive/MyDrive/XAI4Traj/models"

model_path = f"{model_dir}/{model_str}_{dataset_str}.joblib"

model = None
if os.path.exists(model_path):
    model = load(model_path)
    print(f"Model loaded: {model_path}")
else:
    print(f"Model file not found: {model_path}")

Model loaded: drive/MyDrive/XAI4Traj/models/lstm_geolife.joblib


In [23]:
# make sure the persisted model works
evaluation = model.evaluate(test)
evaluation.show()

59/59 ━━━━━━━━━━━━━━━━━━━━ 12s 196ms/step

General statistics:

Accuracy: 0.446
F1-score: 0.181
Mean precision: 0.206
Mean recall: 0.203

Confusion matrix:

airplane  bike      boat      bus       car       run       subway    taxi      train     walk      precision 
0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       
0.0       5.18      0.0       2.19      0.0       0.0       4.27      9.8       6.06      0.78      34.04     
0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       
33.33     49.19     100.0     40.71     11.61     0.0       16.24     21.57     0.0       14.92     31.24     
66.67     5.83      0.0       19.67     54.19     0.0       11.97     10.78     9.09      10.12     29.79     
0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       0.0       
0.0       1.29      0.0       1.09      0.0       0.0       5.98  

In [24]:
import logging

logging.disable(logging.DEBUG)
# logging.disable(logging.NOTSET)

In [25]:
# sliding window currently not working for RF
segment_funcs = [rdp_segmentation, mdl_segmentation]
# time currently doesn't work with GAN and rotation, and scaling somehow creates overflow issues with the RF featurizer
perturbation_funcs = [gaussian_perturbation]

log_dir = "drive/MyDrive/XAI4Traj/logs"
log_idx = 1
# surrogate_params = {'C': 0.5, 'tol': 0.00001, 'max_iter': 500, 'class_weight': 'balanced'}

In [26]:
# without time
run_experiments(test, segment_funcs, perturbation_funcs, model, time=False, log_dir=log_dir,
                log_dataset=dataset_str, log_model=model_str, log_idx=log_idx)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_1_results.csv: [0, 'traj_bab3a0e5', 0, 0.0, 'ok']
2025-12-14 02:29:40 - Total RAM: 52.96 GB | Used: 9.5%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_1_results.csv: [1, 'traj_f0756078', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_1_results.csv: [2, 'traj_08b42525', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_1_results.csv: [3, 'traj_99feffe0', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


...

[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_mdl_gaussian_1_results.csv: [213, 'traj_f3c62d73', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_mdl_gaussian_1_results.csv: [214, 'traj_7c9d1660', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_mdl_gaussian_1_results.csv: [215, 'traj_1c7586e2', 0, 0.0, 'ok']
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 148ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_mdl_gaussian_1_results.csv: [216, 'traj_62690ba1', 0, 0.0, 'ok']


KeyboardInterrupt: 

In [27]:
# with time
run_experiments(test, segment_funcs, perturbation_funcs, model, time=True, log_dir=log_dir,
                log_dataset=dataset_str, log_model=model_str, log_idx=log_idx)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [0, 'traj_bab3a0e5', 0, 0.0, 'ok']
2025-12-14 02:50:27 - Total RAM: 52.96 GB | Used: 11.1%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [1, 'traj_f0756078', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 90ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [2, 'traj_08b42525', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [3, 'traj_99feffe0', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


...

[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [915, 'traj_c5b654fa', 0, 0.0, 'ok']
16/16 ━━━━━━━━━━━━━━━━━━━━ 2s 104ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [916, 'traj_98c41542', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [917, 'traj_d91469e6', 0, 0.0, 'ok']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step


[INFO] Row saved to drive/MyDrive/XAI4Traj/logs/geolife_lstm_rdp_gaussian_time_1_results.csv: [918, 'traj_51b246a9', 0, 0.0, 'ok']
32/32 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step


KeyboardInterrupt: 

In [28]:
# curr = "01"
# segment_idx = int(curr[0])
# perturb_idx = 0
# has_time = int(curr[1])

def get_results(segment_idx, perturb_idx, has_time):
    segment_str = str(segment_funcs[segment_idx].__name__).rsplit("_", 1)[0]
    perturb_str = str(perturbation_funcs[perturb_idx].__name__).rsplit("_", 1)[0]

    file_path = os.path.join(
        log_dir,
        f"{dataset_str}_{model_str}_{segment_str}_{perturb_str}_{'time_' if has_time else ''}{f'{log_idx}_' if log_idx else ''}results.csv",
    )

    count = 0
    total_change = 0
    avg_precision = 0
    with open(file_path, 'r') as f:
        for i, line in enumerate(f):
            if i > 0 and len(line) > 0:
                count += 1
                _, _, change, precision, status = line.strip().split(',')
                if status == 'ok':
                    total_change += int(change)
                    avg_precision += float(precision)
    recall = total_change / count
    avg_precision = (avg_precision / total_change) if total_change != 0 else 0.0
    h_mean = ((2 * avg_precision * recall) / (avg_precision + recall)) if total_change != 0 else 0.0
    # print(f"{file_path}\nRecall: {recall:.3f}\nPrecision: {avg_precision:.3f}\nF1 Score: {h_mean:.3f}")
    print(f"{recall:.3f}\t{avg_precision:.3f}\t{h_mean:.3f}")

In [29]:
# Gaussian, with/without time
for t in range(2):
    for seg in range(2):
        get_results(seg, 0, t)

0.002	1.000	0.003
0.005	1.000	0.009
0.012	1.000	0.024


FileNotFoundError: [Errno 2] No such file or directory: 'drive/MyDrive/XAI4Traj/logs/geolife_lstm_mdl_gaussian_time_1_results.csv'